In [ ]:
%pip install pydantic_settings langchain langchain-core langchain-google-genai

### IVF Embryo Grading

In [ ]:
import base64
import json

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
# Process the image to base64 so it can be sent to the LLM

image_path = "./1.jpg"

with open(image_path, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
# Load environment variables

from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    GOOGLE_API_KEY: str
    model_config = SettingsConfigDict(env_file=".env")

env = Settings()

In [ ]:
# Initialize the LLM with the Google Generative AI model

llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        api_key=env.GOOGLE_API_KEY,
    )

In [ ]:
system_prompt = """
You are an expert AI assistant for pharmacy logistics. Your task is to analyze an image of medication packaging to identify, count, and verify the items against a known database.

Follow this process step-by-step:

### Step 1: Image Quality Assessment
- Analyze the overall image. Is it clear or blurry?
- Is there significant glare or shadow obscuring the text?
- Are the packages fully visible or partially cut off?
- Is the image rotation or perspective correct?

### Step 2: Object Detection and Counting
- Identify how many distinct boxes/packages of the same medication are clearly visible in the image.
- Provide a final integer count.

### Step 3: Text Extraction (OCR) from the Clearest Package
- Carefully read all the text from the most visible and clear medication package.
- Extract the following specific pieces of information:
  - Brand Name (e.g., Brintellix, Tylenol)
  - Active Ingredient / Generic Name (e.g., Vortioxetine, Paracetamol)
  - Dosage (e.g., 10 mg, 500 mg)
  - Package Quantity (e.g., 28 tablets)

### Step 4: Verification Against Database
- Compare the extracted "Brand Name" and "Active Ingredient" against the `KNOWN MEDICATION DATABASE` provided below.
- Your goal is to find a match. For example, if you read "Bactesyn 1.5g", you should match it to "BACTESYN 1.5 GRAM INJ".
- If a match is found, extract its corresponding `item_code` and full `item_name`.
- If no reasonable match is found, the fields for the database match must be null.

### KNOWN MEDICATION DATABASE:
{medication_database}

### OUTPUT FORMAT:
Provide your final assessment in a single, clean JSON object. Do not include the step-by-step analysis in the final JSON.

```json
{{
  "image_assessment": {{
    "clarity": "string",
    "lighting": "string",
    "framing": "string"
  }},
  "detected_medication": {{
    "brand_name": "string | null",
    "active_ingredient": "string | null",
    "dosage": "string | null"
  }},
  "inventory": {{
      "count": "integer",
      "package_quantity": "string | null"
  }},
  "database_match": {{
    "item_code": "string | null",
    "item_name": "string | null"
  }},
  "summary": "string"
}}
```

Now analyze the provided medicine image following this systematic chain-of-thought approach and give me the JSON output.
"""

In [ ]:
# Create the prompt for the LLM 

prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", [
            {
                "type": "text",
                "text": "Please analyze this image according to the instructions."
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                }
            }
        ])
    ])

In [ ]:
# Create the chain to invoke the LLM

chain = prompt | llm
response = chain.invoke({})
response.content

In [ ]:
def print_readable_result(response_text: str) -> None:
    """Parses the LLM's JSON response and prints a user-friendly report."""
    print("=" * 60)
    print("💊 PHARMACY INVENTORY ANALYSIS REPORT")
    print("=" * 60)

    try:
        # Clean up the response to extract only the JSON
        json_str = response_text.strip().lstrip("```json").rstrip("```").strip()
        result = json.loads(json_str)

        # Image Assessment
        assessment = result.get('image_assessment', {})
        print("\n📷 IMAGE ASSESSMENT:")
        print(f"   - Clarity: {assessment.get('clarity', 'N/A')}")
        print(f"   - Lighting: {assessment.get('lighting', 'N/A')}")
        print(f"   - Framing: {assessment.get('framing', 'N/A')}")
        print(f"   - Rotation: {assessment.get('rotation', 'N/A')}")

        # Detected Medication
        med = result.get('detected_medication', {})
        print("\n🔬 DETECTED MEDICATION:")
        print(f"   - Brand Name: {med.get('brand_name', 'N/A')}")
        print(f"   - Active Ingredient: {med.get('active_ingredient', 'N/A')}")
        print(f"   - Dosage: {med.get('dosage', 'N/A')}")

        # Inventory
        inv = result.get('inventory', {})
        print("\n📦 INVENTORY:")
        print(f"   - Count in Image: {inv.get('count', 'N/A')}")
        print(f"   - Quantity per Box: {inv.get('package_quantity', 'N/A')}")

        # Database Verification
        match = result.get('database_match', {})
        print("\n🔍 DATABASE VERIFICATION:")
        if match and match.get('item_code'):
            print(f"   - Status: ✅ Match Found")
            print(f"   - Item Code: {match.get('item_code')}")
            print(f"   - Item Name: {match.get('item_name')}")
        else:
            print(f"   - Status: ⚠️ Not found in database.")

        # Summary
        print("\n📋 SUMMARY:")
        print(f"   {result.get('summary', 'No summary provided.')}")

    except (json.JSONDecodeError, AttributeError) as e:
        print(f"\n❌ Error parsing the response: {e}")
        print("\n🤖 RAW RESPONSE:")
        print("-" * 40)
        print(response_text)
    
    print("\n" + "=" * 60)

In [ ]:
print_readable_result(response.content)